<a href="https://colab.research.google.com/github/hasanahmad676/Image_segmentation/blob/main/watershed_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the pre-trained U-Net model
model_path= tf.keras.models.load_model("water_segmentation_model.h5")

# Load the pre-trained U-Net model
model = tf.keras.models.load_model(model_path)

# Load and preprocess the input image
input_image_path = "/content/drive/MyDrive/OralCnacer/Normal"
input_image = cv2.imread(input_image_path)
input_image = cv2.resize(input_image, (256, 256))  # Resize the image to match the model input size
input_image = input_image / 255.0  # Normalize the image to [0, 1]

# Perform segmentation
segmentation_mask = model.predict(np.expand_dims(input_image, axis=0))[0]  # Predict the mask for the input image

# Apply a threshold to the segmentation mask to obtain a binary mask
threshold = 0.5  # Adjust the threshold as needed
binary_mask = (segmentation_mask > threshold).astype(np.uint8) * 255

# Save the segmentation mask as an image
cv2.imwrite("/content/drive/MyDrive/OralCnacer/seg", binary_mask)


OSError: ignored

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import os  # Import the 'os' module for working with file paths

# Get the current working directory
current_directory = os.getcwd()

# Specify the absolute path to your pre-trained U-Net model
model_path = os.path.join(current_directory, "path/to/your/model/water_segmentation_model.h5")

# Load the pre-trained U-Net model
model = tf.keras.models.load_model(model_path)

# Load and preprocess the input image
input_image_path = os.path.join(current_directory, "/content/drive/MyDrive/OralCnacer/Normal")
input_image = cv2.imread(input_image_path)
input_image = cv2.resize(input_image, (256, 256))  # Resize the image to match the model input size
input_image = input_image / 255.0  # Normalize the image to [0, 1]

# Perform segmentation
segmentation_mask = model.predict(np.expand_dims(input_image, axis=0))[0]  # Predict the mask for the input image

# Apply a threshold to the segmentation mask to obtain a binary mask
threshold = 0.5  # Adjust the threshold as needed
binary_mask = (segmentation_mask > threshold).astype(np.uint8) * 255

# Save the segmentation mask as an image
cv2.imwrite("/content/drive/MyDrive/OralCnacer/seg", binary_mask)


OSError: ignored

In [ ]:
!pip install opencv-python


In [ ]:
import cv2
import numpy as np
import os

# Path to the directory containing your dataset of oral cancer images
dataset_dir = '/content/drive/MyDrive/OralCnacer/Normal'

# List all image files in the dataset directory
image_files = [f for f in os.listdir(dataset_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Process each image in the dataset
for image_file in image_files:
    # Load the histopathological oral cancer image
    image_path = os.path.join(dataset_dir, image_file)
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image to create a binary mask
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Perform morphological operations to clean up the mask
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Label markers for watershed algorithm
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    # Apply watershed algorithm
    cv2.watershed(image, markers)
    image[markers == -1] = [0, 0, 255]  # Mark watershed boundaries in red

    # Save or display the segmented image
    segmented_image_path = os.path.join(dataset_dir, f'segmented_{image_file}')
    cv2.imwrite(segmented_image_path, image)


KeyboardInterrupt: ignored

In [ ]:
import cv2
import numpy as np
import os

# Path to the directory containing your dataset of oral cancer images
dataset_dir = '/content/drive/MyDrive/OralCnacer/Normal'

# Specify the output directory for segmented images
output_dir = '/content/drive/MyDrive/OralCnacer/seg'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# List all image files in the dataset directory
image_files = [f for f in os.listdir(dataset_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Process each image in the dataset
for image_file in image_files:
    # Load the histopathological oral cancer image
    image_path = os.path.join(dataset_dir, image_file)
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image to create a binary mask
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Perform morphological operations to clean up the mask
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Label markers for watershed algorithm
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    # Apply watershed algorithm
    cv2.watershed(image, markers)
    image[markers == -1] = [0, 0, 255]  # Mark watershed boundaries in red

    # Save the segmented image in the output directory
    segmented_image_path = os.path.join(output_dir, f'segmented_{image_file}')
    cv2.imwrite(segmented_image_path, image)
